In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DataFields import DataFields
from DataFields import DateReportedFields
from ProjectFunctions import one_hot_encode_vascular_problems

In [2]:
df_diagnosed = pd.read_csv("diagnosed_imputed.csv")
df_undiagnosed = pd.read_csv("undiagnosed.csv")

In [3]:
df_merged = pd.concat([df_diagnosed, df_undiagnosed], ignore_index=True, sort=False)

df_merged = one_hot_encode_vascular_problems(df_merged)
df_merged = df_merged.drop(columns=["High Blood Pressure"])

df_merged.to_csv("dataset_merged.csv", index=False)

df_merged.describe()

,Birth Year,Sex,Education,Primary Hypertension,Secondary Hypertension,BMI Impedance,Smoking Status,Ever Smoked,Alcohol Intake Frequency,Report of stroke,...,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Has Vascular Dementia,Heart Attack,Angina,Stroke
count,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,...,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000
mean,1943.607732,0.616843,1.619087,0.376754,0.001477,28.506288,0.648876,0.665107,2.928308,0.204383,...,0.064741,1.428748,7.194437,83.549278,148.858045,70.335019,0.477469,0.097020,0.081015,0.062300
std,4.804711,0.486216,1.057314,0.484632,0.038414,4.794312,0.648016,0.472011,1.642911,0.403300,...,0.030105,0.674356,1.952232,10.856766,19.917436,12.742064,0.499554,0.296022,0.272891,0.241729
min,1937.000000,0.000000,0.000000,0.000000,0.000000,15.757600,0.000000,0.000000,1.000000,0.000000,...,0.010000,0.210000,2.370000,47.000000,78.000000,35.000000,0.000000,0.000000,0.000000,0.000000
25%,1940.000000,0.000000,1.000000,0.000000,0.000000,25.292600,0.000000,0.000000,2.000000,0.000000,...,0.047000,1.040000,5.900000,76.000000,135.000000,61.000000,0.000000,0.000000,0.000000,0.000000
50%,1942.000000,1.000000,2.000000,0.000000,0.000000,27.797900,1.000000,1.000000,3.000000,0.000000,...,0.061000,1.346000,7.000000,83.000000,148.000000,69.000000,0.000000,0.000000,0.000000,0.000000
75%,1946.000000,1.000000,3.000000,1.000000,0.000000,31.103400,1.000000,1.000000,4.000000,0.000000,...,0.079000,1.728000,8.270000,90.000000,161.000000,78.000000,1.000000,0.000000,0.000000,0.000000
max,1965.000000,1.000000,3.000000,1.000000,1.000000,59.188300,2.000000,1.000000,6.000000,1.000000,...,1.077000,25.278000,34.130000,133.000000,241.000000,169.000000,1.000000,1.000000,1.000000,1.000000


#### Identify and drop highly correlated features by pearson correlation

In [4]:
def print_highly_correlated_features(df, threshold=0.8):
    corr_matrix = df.corr(method="pearson")
    correlated_features = {}

    for col in corr_matrix.columns:
        high_corr = corr_matrix[col][(corr_matrix[col] >= threshold) & (corr_matrix[col] < 1)].index.tolist()
        if high_corr:
            correlated_features[col] = high_corr
    print("Highly correlated features:")
    for feature, related_features in correlated_features.items():
        print(f"{feature}: {', '.join(related_features)}")

def drop_correlated_features(df, threshold=0.8, keep_list=None):
    if keep_list is None:
        keep_list = []
    
    corr_matrix = df.corr(method="pearson").abs()
    to_drop = set()
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):  # Avoid duplicate checks
            feature1 = corr_matrix.columns[i]
            feature2 = corr_matrix.columns[j]
            
            if corr_matrix.iloc[i, j] > threshold:
                if feature1 in keep_list and feature2 in keep_list:
                    raise ValueError(f"Cannot drop either '{feature1}' or '{feature2}' as both are in keep_list.")
                
                if feature1 in keep_list:
                    to_drop.add(feature2)  # Drop feature2 if feature1 is in keep_list
                elif feature2 in keep_list:
                    to_drop.add(feature1)
                else:
                    to_drop.add(feature2)
    
    # Drop identified features
    df = df.drop(columns=to_drop, errors="ignore")
    
    print("Features dropped:")
    for x in to_drop:
        print(x)
            
    return df

#### View the correlation matrix

In [5]:
df_merged.corr()

,Birth Year,Sex,Education,Primary Hypertension,Secondary Hypertension,BMI Impedance,Smoking Status,Ever Smoked,Alcohol Intake Frequency,Report of stroke,...,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Has Vascular Dementia,Heart Attack,Angina,Stroke
Birth Year,1.000000,-0.004045,0.078625,-0.082066,0.020489,0.077393,-0.026514,-0.046862,0.024667,-0.022297,...,0.057457,0.042823,0.002602,0.141451,-0.093448,0.051096,-0.087266,-0.026053,-0.060290,0.006202
Sex,-0.004045,1.000000,0.086775,-0.002895,0.030317,0.007130,0.152235,0.163031,-0.208191,0.055293,...,0.070809,-0.002776,0.059512,0.055129,-0.002654,-0.061954,-0.049749,0.143685,0.048373,0.058549
Education,0.078625,0.086775,1.000000,-0.322254,-0.034654,-0.081536,-0.106684,-0.026149,-0.173081,-0.106834,...,-0.049295,-0.062505,-0.106473,0.077487,0.010198,-0.058312,-0.380192,-0.058439,-0.000905,-0.012054
Primary Hypertension,-0.082066,-0.002895,-0.322254,1.000000,0.049474,0.094553,0.078304,0.031641,0.131814,0.268789,...,0.039291,0.056477,0.087217,-0.055778,0.004266,0.055221,0.813361,0.071351,0.026163,0.077121
Secondary Hypertension,0.020489,0.030317,-0.034654,0.049474,1.000000,0.020223,0.040634,0.013711,0.021192,0.012301,...,0.019500,0.025582,0.008353,-0.001946,0.007356,-0.033216,0.040241,0.052371,-0.011421,-0.009915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pulse Rate at Blood Pressure,0.051096,-0.061954,-0.058312,0.055221,-0.033216,0.115548,0.016086,0.000382,0.062441,0.016247,...,0.074733,0.065797,0.174453,0.265804,0.033077,1.000000,0.045811,-0.124910,-0.070468,0.037050
Has Vascular Dementia,-0.087266,-0.049749,-0.380192,0.813361,0.040241,0.006388,0.082869,0.021265,0.149715,0.253923,...,-0.007481,0.016730,0.074923,-0.125862,-0.083751,0.045811,1.000000,0.034773,-0.021839,0.057520
Heart Attack,-0.026053,0.143685,-0.058439,0.071351,0.052371,0.054772,0.077543,0.033401,0.056466,0.069058,...,0.050409,0.049168,0.071531,-0.161732,-0.144297,-0.124910,0.034773,1.000000,-0.097324,-0.084490
Angina,-0.060290,0.048373,-0.000905,0.026163,-0.011421,0.046865,0.012088,0.027114,0.041526,0.017362,...,0.017625,0.016133,0.024758,-0.131580,-0.069365,-0.070468,-0.021839,-0.097324,1.000000,-0.076531


#### Print highly correlated features and drop those excluded from the keey_list

In [7]:
print_highly_correlated_features(df_merged, threshold=0.82)

df_merged = drop_correlated_features(df_merged, threshold = 0.82, keep_list=["Has Vascular Dementia", "Sex"])

Highly correlated features:
Sex: Testosterone
Apolipoprotein A: HDL cholesterol
Apolipoprotein B: Cholesterol, LDL direct
Cholesterol: Apolipoprotein B, LDL direct
Direct bilirubin: Total bilirubin
HDL cholesterol: Apolipoprotein A
LDL direct: Apolipoprotein B, Cholesterol
Testosterone: Sex
Total bilirubin: Direct bilirubin
Basophil count: Basophil percentage
Basophil percentage: Basophil count
Eosinophil count: Eosinophil percentage
Eosinophil percentage: Eosinophil count
Haematocrit percentage: Haemoglobin concentration, Red blood cell (erythrocyte) count
Haemoglobin concentration: Haematocrit percentage
High light scatter reticulocyte count: High light scatter reticulocyte percentage
High light scatter reticulocyte percentage: High light scatter reticulocyte count
Mean corpuscular haemoglobin: Mean corpuscular volume
Mean corpuscular volume: Mean corpuscular haemoglobin
Neutrophil count: White blood cell (leukocyte) count
Nucleated red blood cell count: Nucleated red blood cell perc

#### Save the new dataframe

In [8]:
df_merged.to_csv("vascular_dementia_dataset.csv", index=False)
df_merged.describe()

,Birth Year,Sex,Education,Primary Hypertension,Secondary Hypertension,BMI Impedance,Smoking Status,Ever Smoked,Alcohol Intake Frequency,Report of stroke,...,Platelet distribution width,Red blood cell (erythrocyte) distribution width,Reticulocyte count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Has Vascular Dementia,Heart Attack,Angina,Stroke
count,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,...,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000,4061.000000
mean,1943.607732,0.616843,1.619087,0.376754,0.001477,28.506288,0.648876,0.665107,2.928308,0.204383,...,16.532465,13.612258,0.064741,83.549278,148.858045,70.335019,0.477469,0.097020,0.081015,0.062300
std,4.804711,0.486216,1.057314,0.484632,0.038414,4.794312,0.648016,0.472011,1.642911,0.403300,...,0.528219,1.066313,0.030105,10.856766,19.917436,12.742064,0.499554,0.296022,0.272891,0.241729
min,1937.000000,0.000000,0.000000,0.000000,0.000000,15.757600,0.000000,0.000000,1.000000,0.000000,...,13.710000,11.490000,0.010000,47.000000,78.000000,35.000000,0.000000,0.000000,0.000000,0.000000
25%,1940.000000,0.000000,1.000000,0.000000,0.000000,25.292600,0.000000,0.000000,2.000000,0.000000,...,16.180000,13.000000,0.047000,76.000000,135.000000,61.000000,0.000000,0.000000,0.000000,0.000000
50%,1942.000000,1.000000,2.000000,0.000000,0.000000,27.797900,1.000000,1.000000,3.000000,0.000000,...,16.490000,13.430000,0.061000,83.000000,148.000000,69.000000,0.000000,0.000000,0.000000,0.000000
75%,1946.000000,1.000000,3.000000,1.000000,0.000000,31.103400,1.000000,1.000000,4.000000,0.000000,...,16.830000,14.000000,0.079000,90.000000,161.000000,78.000000,1.000000,0.000000,0.000000,0.000000
max,1965.000000,1.000000,3.000000,1.000000,1.000000,59.188300,2.000000,1.000000,6.000000,1.000000,...,19.400000,37.190000,1.077000,133.000000,241.000000,169.000000,1.000000,1.000000,1.000000,1.000000
